In [12]:
%pip install numpy
%pip install pandas 
%pip install matplotlib
%pip install qiskit qiskit-aer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# quantum_scheduler_demo_aer_fast.py
# Seminar-ready, Aer-backed, optimized for speed on laptops.
# Generates: decision_boundary.png (and a quick decision_boundary_smoke.png),
#            adaptation_cumulative.png, adaptation_modes.png, nvqlink_latency_demo.png

import os, math, numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import lru_cache
from time import perf_counter

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error
from qiskit.circuit import Delay
from qiskit.transpiler import InstructionDurations
try:
    from qiskit_aer.library import SaveStatevector
except Exception:
    SaveStatevector = None

# ------------------------------
# Presets (flip FAST=False for fuller/denser plots)
# ------------------------------
FAST = True
SEED = 7                 # global seed (deterministic circuits + simulator)
SMOOTHING_REPS = 3       # light averaging of fidelity to kill tiny jitter (set 1 to disable)

if FAST:
    N_QUBITS = 3
    DEPTH = 24
    P2Q_GRID = np.linspace(0.004, 0.016, 12)       # 12 x 14 = 168 points
    L_GRID   = np.logspace(-6, -3, 14)
else:
    N_QUBITS = 4
    DEPTH = 64
    P2Q_GRID = np.linspace(0.002, 0.02, 40)        # 40 x 40 = 1600 points
    L_GRID   = np.logspace(-6, -2, 40)

OUTDIR = "./artifacts"
os.makedirs(OUTDIR, exist_ok=True)

# ------------------------------
# Utility model
# U = α * Fidelity - β * Latency - γ * Shots
# ------------------------------
ALPHA, BETA, GAMMA = 1.0, 1.0, 1e-4
def utility(fid, lat, sh): return ALPHA * float(fid) - BETA * float(lat) - GAMMA * float(sh)
def clip01(x): return max(0.0, min(1.0, float(x)))
def _q(x, dec=6): return round(float(x), dec)  # quantize for stable cache keys

# ------------------------------
# Timing/Noise model (IBM-like realistic, but compact)
# ------------------------------
DT = 2e-10                # 0.2 ns (device-level)
T1 = 100e-6               # 100 µs
T2 = 80e-6                # 80 µs
U1 = 50e-9                # 50 ns
U2 = 60e-9                # 60 ns
CX = 400e-9               # 400 ns
MEAS = 3e-6               # measurement ~3 µs (dominant)
RESET = 1e-6              # 1 µs
CLASSICAL_BOUNDARY = 2e-6 # stitching/classical processing per boundary

# Switch: thermal-relaxation on gates (slower). Keep False for speed.
USE_TRE_ON_GATES = False

# Instruction durations for scheduling (used if your Qiskit supports it)
DURATIONS = InstructionDurations([
    ("rz", 1, U1), ("rx", 1, U2),
    ("cx", 2, CX), ("measure", 1, MEAS), ("reset", 1, RESET)
])

# Single global simulators (seeded for determinism)
SIM_SV = AerSimulator(method='statevector')
SIM_SV.set_options(seed_simulator=SEED)
# You can switch fidelity paths to density_matrix for even more stability (slower):
# SIM_DM = AerSimulator(method='density_matrix'); SIM_DM.set_options(seed_simulator=SEED)

# ------------------------------
# Helpers: Aer statevector extraction (portable across Aer versions)
# ------------------------------
def _sv_from_aer(qc: QuantumCircuit, noise_model: NoiseModel = None):
    """Run circuit on Aer (statevector) and return the final statevector."""
    qc2 = qc.copy()
    # Ensure a "save_statevector" instruction exists for all Aer versions
    try:
        qc2.save_statevector()
    except Exception:
        if SaveStatevector is not None:
            qc2.append(SaveStatevector(), [])
        else:
            raise RuntimeError("Your qiskit-aer lacks save_statevector(); please update qiskit-aer.")
    res = SIM_SV.run(qc2, noise_model=noise_model, seed_simulator=SEED).result()
    try:
        return res.get_statevector(qc2)
    except Exception:
        data0 = res.data(0)
        if 'statevector' in data0:
            return data0['statevector']
        raise RuntimeError("No statevector found in result; check Aer version.")

# ------------------------------
# Caching helpers
# ------------------------------
@lru_cache(maxsize=None)
def _noise_model_cached(p2q: float, n_qubits: int) -> NoiseModel:
    """
    Build and cache a noise model based on 2-qubit depolarizing noise p2q.
    1q depolarization set as p2q/10 (capped). Optional TRE on gates.
    """
    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(depolarizing_error(float(p2q), 2), ['cx'])
    nm.add_all_qubit_quantum_error(depolarizing_error(min(0.1*float(p2q), 0.02), 1), ['rx','rz'])
    if USE_TRE_ON_GATES:
        nm.add_all_qubit_quantum_error(thermal_relaxation_error(T1, T2, U1), ['rx','rz'])
        tre2 = thermal_relaxation_error(T1, T2, CX).tensor(thermal_relaxation_error(T1, T2, CX))
        nm.add_all_qubit_quantum_error(tre2, ['cx'])
    return nm

@lru_cache(maxsize=None)
def _layered_cached(n_qubits: int, depth: int, seed: int = SEED) -> QuantumCircuit:
    """Fixed-random layered circuit: RX/RZ on all, CX in a line pattern."""
    rng = np.random.default_rng(seed)   # fixed seed → same circuit everywhere
    qc = QuantumCircuit(int(n_qubits))
    for _ in range(int(depth)):
        for q in range(int(n_qubits)):
            qc.rx(float(rng.uniform(-np.pi, np.pi)), q)
            qc.rz(float(rng.uniform(-np.pi, np.pi)), q)
        for q in range(0, int(n_qubits) - 1, 2):
            qc.cx(q, q + 1)
        for q in range(1, int(n_qubits) - 1, 2):
            qc.cx(q, q + 1)
    return qc

@lru_cache(maxsize=None)
def _ideal_sv_cached(n_qubits: int, depth: int) -> Statevector:
    return Statevector.from_instruction(_layered_cached(int(n_qubits), int(depth)))

# ---- CUT CIRCUITS (allocate cbits so measure() is valid) ----
@lru_cache(maxsize=None)
def _cut_fidelity_circuit(n_qubits: int, depth: int, cuts: int) -> QuantumCircuit:
    assert cuts == 1
    n = int(n_qubits)
    seg_d = max(1, int(depth)//2)
    qc = QuantumCircuit(n, n)  # classical bits present
    qc.compose(_layered_cached(n, seg_d), inplace=True)

    boundary = [0, 1] if n >= 2 else [0]
    qc.measure(boundary, boundary)

    # classical stitching delay on all qubits
    for q in range(n):
        qc.append(Delay(int(CLASSICAL_BOUNDARY / DT)), [q])

    # reset boundary qubits before continuing
    for q in boundary:
        qc.reset(q)

    qc.compose(_layered_cached(n, int(depth) - seg_d), inplace=True)
    return qc

@lru_cache(maxsize=None)
def _cut_latency_circuit(n_qubits: int, depth: int, cuts: int) -> QuantumCircuit:
    qc = _cut_fidelity_circuit(n_qubits, depth, cuts).copy()
    qc.measure_all()  # include measurement cost in timing
    return qc

# ---- TELEPORT CIRCUITS ----
def _teleport_block() -> QuantumCircuit:
    """
    3-qubit teleportation block:
      q0=data, q1=Alice, q2=Bob; 2 classical bits store Bell outcomes.
      Corrections (CX/CZ) stand in for classically conditioned X/Z.
    """
    qct = QuantumCircuit(3, 2)
    # Bell pair between Alice (1) and Bob (2)
    qct.h(1); qct.cx(1, 2)
    # Bell measurement on data+Alice
    qct.cx(0, 1); qct.h(0)
    qct.measure(0, 0); qct.measure(1, 1)
    # Symbolic feed-forward corrections
    qct.cx(0, 2); qct.cz(1, 2)
    return qct

@lru_cache(maxsize=None)
def _teleport_fidelity_circuit(n_qubits: int, depth: int, L_s_q: float) -> QuantumCircuit:
    nq = max(int(n_qubits), 3)
    seg_d = max(1, int(depth)//2)
    qc = QuantumCircuit(nq, 2)
    qc.compose(_layered_cached(nq, seg_d), inplace=True)
    # explicit link latency (idle) on all qubits
    for q in range(nq):
        qc.append(Delay(int(float(L_s_q) / DT)), [q])
    qct = _teleport_block()
    qc.compose(qct, qubits=[0,1,2], clbits=[0,1], inplace=True)
    qc.compose(_layered_cached(nq, int(depth) - seg_d), inplace=True)
    return qc

@lru_cache(maxsize=None)
def _teleport_latency_circuit(n_qubits: int, depth: int, L_s_q: float) -> QuantumCircuit:
    qc = _teleport_fidelity_circuit(n_qubits, depth, L_s_q).copy()
    # Extra ~1 µs for classical feed-forward/corrections
    for q in range(max(int(n_qubits), 3)):
        qc.append(Delay(int(1e-6 / DT)), [q])
    qc.measure_all()
    return qc

# ---- TRANSPILE (with scheduling if available) ----
@lru_cache(maxsize=None)
def _transpile_cached(qc_key: tuple) -> QuantumCircuit:
    """
    Keyed by (mode, n_qubits, depth, cuts, Ls). Falls back if durations unsupported.
    """
    mode, n_qubits, depth, cuts, Ls = qc_key
    if mode == "single":
        qc = _layered_cached(n_qubits, depth).copy()
        qc.measure_all()
    elif mode == "cut_fid":
        qc = _cut_fidelity_circuit(n_qubits, depth, cuts)
    elif mode == "cut_lat":
        qc = _cut_latency_circuit(n_qubits, depth, cuts)
    elif mode == "tel_fid":
        qc = _teleport_fidelity_circuit(n_qubits, depth, Ls)
    elif mode == "tel_lat":
        qc = _teleport_latency_circuit(n_qubits, depth, Ls)
    else:
        raise ValueError("Unknown mode for transpile cache")

    try:
        return transpile(
            qc, SIM_SV, optimization_level=0,
            scheduling_method='alap', instruction_durations=DURATIONS
        )
    except TypeError:
        # Older Qiskit without instruction_durations
        return transpile(qc, SIM_SV, optimization_level=0)

# ------------------------------
# Duration accounting
# ------------------------------
def circuit_duration_seconds_from_scheduled(scheduled: QuantumCircuit) -> float:
    dur = 0.0
    for inst, qargs, _ in scheduled.data:
        name = inst.name
        if name in ("rx", "rz"): dur += U1
        elif name == "cx":       dur += CX
        elif name == "measure":  dur += MEAS
        elif name == "reset":    dur += RESET
        elif name == "delay" and getattr(inst, "duration", None) is not None:
            dur += inst.duration * DT
    return dur

# ------------------------------
# Averaging helper (tiny smoothing)
# ------------------------------
def _avg_fidelity(eval_once, reps: int = SMOOTHING_REPS) -> float:
    if reps <= 1:
        return float(eval_once())
    acc = 0.0
    for _ in range(reps):
        acc += float(eval_once())
    return acc / reps

# ------------------------------
# Aer fidelity wrappers (use caches + averaging)
# ------------------------------
def fidelity_single_aer(p2q: float, depth: int, n_qubits: int = N_QUBITS) -> float:
    nm = _noise_model_cached(_q(p2q), n_qubits)
    tqc = _transpile_cached(("single", n_qubits, depth, 0, 0.0))
    sv_ideal = _ideal_sv_cached(n_qubits, depth)
    def once():
        sv_noisy = _sv_from_aer(tqc, noise_model=nm)
        return state_fidelity(sv_ideal, sv_noisy)
    return clip01(_avg_fidelity(once))

def fidelity_cut_aer(p2q: float, depth: int, cuts: int = 1, n_qubits: int = N_QUBITS) -> float:
    assert cuts == 1
    nm = _noise_model_cached(_q(p2q), n_qubits)
    tqc = _transpile_cached(("cut_fid", n_qubits, depth, cuts, 0.0))
    sv_ideal = _ideal_sv_cached(n_qubits, depth)
    def once():
        sv_noisy = _sv_from_aer(tqc, noise_model=nm)
        return state_fidelity(sv_ideal, sv_noisy)
    return clip01(_avg_fidelity(once))

def fidelity_teleport_aer(p2q: float, depth: int, L_s: float, n_qubits: int = N_QUBITS) -> float:
    p = _q(p2q, 6); Ls = _q(L_s, 9)
    nm = _noise_model_cached(p, max(n_qubits, 3))
    tqc = _transpile_cached(("tel_fid", max(n_qubits,3), depth, 0, float(Ls)))
    sv_ideal = _ideal_sv_cached(max(n_qubits,3), depth)
    def once():
        sv_noisy = _sv_from_aer(tqc, noise_model=nm)
        return state_fidelity(sv_ideal, sv_noisy)
    # Apply analytical dephasing penalty for link idle (fast)
    return clip01(_avg_fidelity(once) * math.exp(-float(Ls) / T2))

# ------------------------------
# Latency wrappers (use caches)
# ------------------------------
def latency_single_aer(depth: int, n_qubits: int = N_QUBITS) -> float:
    tqc = _transpile_cached(("single", n_qubits, depth, 0, 0.0))
    return circuit_duration_seconds_from_scheduled(tqc)

def latency_cut_aer(depth: int, cuts: int = 1, n_qubits: int = N_QUBITS) -> float:
    tqc = _transpile_cached(("cut_lat", n_qubits, depth, cuts, 0.0))
    return circuit_duration_seconds_from_scheduled(tqc)

def latency_teleport_aer(depth: int, L_s: float, n_qubits: int = N_QUBITS) -> float:
    tqc = _transpile_cached(("tel_lat", max(n_qubits,3), depth, 0, _q(L_s,9)))
    return circuit_duration_seconds_from_scheduled(tqc)

# ------------------------------
# Shots (keep simple)
# ------------------------------
def shots_single_aer(depth: int) -> int: return 1000 + 20 * int(depth)
def shots_cut_aer(depth: int, cuts: int = 1) -> int: return int(shots_single_aer(depth) * (1.5 ** cuts))

# Glue to generic names used by plotting / bandit code
def fidelity_single(p2q, depth):     return fidelity_single_aer(p2q, depth)
def latency_single(depth):            return latency_single_aer(depth)
def shots_single(depth):              return shots_single_aer(depth)
def fidelity_cut(p2q, depth, cuts=1): return fidelity_cut_aer(p2q, depth, cuts)
def latency_cut(depth, cuts=1):       return latency_cut_aer(depth, cuts)
def shots_cut(depth, cuts=1):         return shots_cut_aer(depth, cuts)
def fidelity_teleport(p2q, depth, L_s):   return fidelity_teleport_aer(p2q, depth, L_s)
def latency_teleport(depth, L_s):         return latency_teleport_aer(depth, L_s)

# ------------------------------
# Decision boundary heatmap
# ------------------------------
def decision_boundary_grid(depth=DEPTH, p2q_grid=P2Q_GRID, L_grid=L_GRID):
    """
    Evaluate utility for all combinations on the p2q × latency grid.
    Decide best mode at each point.
    """
    rows = []
    for p2q in p2q_grid:
        for L in L_grid:
            U_single = utility(fidelity_single(p2q, depth), latency_single(depth), shots_single(depth))
            U_cut    = utility(fidelity_cut(p2q, depth, 1),   latency_cut(depth, 1),   shots_cut(depth, 1))
            U_tel    = utility(fidelity_teleport(p2q, depth, L), latency_teleport(depth, L), shots_single(depth))
            best = max([("single",U_single),("cut",U_cut),("teleport",U_tel)], key=lambda x:x[1])[0]
            rows.append({"p2q":p2q,"latency_s":L,"U_single":U_single,"U_cut":U_cut,"U_teleport":U_tel,"best_mode":best})
    return pd.DataFrame(rows)

def _majority_filter(Z: np.ndarray, iters: int = 1) -> np.ndarray:
    """Simple 3x3 majority filter to visually denoise isolated pixels (optional)."""
    Z = Z.copy().astype(int)
    H, W = Z.shape
    for _ in range(iters):
        Z2 = Z.copy()
        for i in range(1, H-1):
            for j in range(1, W-1):
                block = Z[i-1:i+2, j-1:j+2].ravel()
                vals, counts = np.unique(block, return_counts=True)
                Z2[i, j] = vals[np.argmax(counts)]
        Z = Z2
    return Z

def plot_decision_boundary(df, path_png, smooth_majority: bool = True):
    """
    Turn decision boundary DataFrame into a heatmap and save to file.
    """
    df = df.copy()
    df["logL"] = np.log10(df["latency_s"])
    pvals = sorted(df["p2q"].unique())
    lvals = sorted(df["logL"].unique())
    Z = np.zeros((len(pvals), len(lvals)), dtype=int)
    enc = {"single":0,"cut":1,"teleport":2}
    for i,p in enumerate(pvals):
        sub = df[df["p2q"]==p]
        for j,l in enumerate(lvals):
            m = sub[sub["logL"]==l]["best_mode"]
            Z[i,j] = enc.get(m.values[0], 0) if len(m) else 0
    if smooth_majority:
        Z = _majority_filter(Z, iters=1)
    plt.figure(figsize=(8,4.5))
    plt.imshow(Z, aspect="auto", origin="lower",
               extent=[min(lvals), max(lvals), min(pvals), max(pvals)])
    plt.colorbar(label="Best mode (0=Single,1=Cut,2=Teleport)")
    plt.xlabel("log10(link latency [s])")
    plt.ylabel("Two-qubit error probability")
    plt.title("Decision Boundary (best mode)")
    plt.tight_layout(); plt.savefig(path_png, dpi=160); plt.close()

# ------------------------------
# Bandit adaptation (unchanged)
# ------------------------------
class TSBandit:
    def __init__(self, arms, init_mean=0.0, init_var=0.5, obs_std=0.01):
        self.arms=arms; self.means={a:init_mean for a in self.arms}; self.vars={a:init_var for a in self.arms}; self.obs_var=obs_std**2
    def select(self):
        samples = {a: np.random.normal(self.means[a], np.sqrt(self.vars[a])) for a in self.arms}
        return max(samples, key=samples.get)
    def update(self, arm, reward):
        m,v=self.means[arm], self.vars[arm]; ov=self.obs_var
        post_var  = 1.0/(1.0/v + 1.0/ov)
        post_mean = post_var*(m/v + reward/ov)
        self.means[arm]=post_mean; self.vars[arm]=post_var

def simulate_true_utils(T=40, segments=((0,10,0.005),(10,20,0.012),(20,30,0.008),(30,40,0.016)), depth=DEPTH, L_s=1e-3):
    rows=[]
    for t in range(T):
        p2q = segments[-1][2]
        for a,b,val in segments:
            if a<=t<b: p2q=val; break
        U_single = utility(fidelity_single(p2q, depth), latency_single(depth), shots_single(depth))
        U_cut    = utility(fidelity_cut(p2q, depth, 1),   latency_cut(depth, 1),   shots_cut(depth, 1))
        U_tel    = utility(fidelity_teleport(p2q, depth, L_s), latency_teleport(depth, L_s), shots_single(depth))
        oracle   = max([("single",U_single),("cut",U_cut),("teleport",U_tel)], key=lambda x:x[1])[0]
        rows.append({"t":t,"p2q":p2q,"U_single":U_single,"U_cut":U_cut,"U_teleport":U_tel,"oracle":oracle})
    return pd.DataFrame(rows)

def run_bandit(df_true, switch_cost=0.001, obs_std=0.01):
    arms=["single","cut","teleport"]; ts=TSBandit(arms, obs_std=obs_std)
    prev=None; cum={"bandit":0.0,"single":0.0,"cut":0.0,"teleport":0.0,"oracle":0.0}
    logs=[]
    for _,r in df_true.iterrows():
        t=int(r["t"]); trueU={"single":r["U_single"],"cut":r["U_cut"],"teleport":r["U_teleport"]}
        cum["oracle"] += trueU[r["oracle"]]
        a = ts.select()
        rew = trueU[a] - (switch_cost if (prev is not None and a!=prev) else 0.0)
        obs = rew + np.random.normal(0.0, obs_std)
        ts.update(a, obs); prev=a; cum["bandit"]+=rew
        cum["single"]+=trueU["single"]; cum["cut"]+=trueU["cut"]; cum["teleport"]+=trueU["teleport"]
        logs.append({"t":t,"chosen":a,"oracle":r["oracle"], **{f"cum_{k}":v for k,v in cum.items()}})
    return pd.DataFrame(logs)

def plot_adaptation(df_true, df_logs, modes_png, cumu_png):
    plt.figure(figsize=(9,3))
    plt.plot(df_true["t"], df_true["p2q"], linewidth=2)
    y=df_true["p2q"].values; mark={"single":"o","cut":"s","teleport":"^"}
    for t, a in zip(df_logs["t"], df_logs["chosen"]):
        plt.scatter([t],[y[t]], s=25, marker=mark.get(a,"o"))
    plt.xlabel("Job index"); plt.ylabel("Two-qubit error probability")
    plt.title("Drifting noise with bandit-chosen mode"); plt.tight_layout()
    plt.savefig(modes_png, dpi=160); plt.close()

    plt.figure(figsize=(9,4))
    plt.plot(df_logs["t"], df_logs["cum_bandit"], linewidth=2, label="Bandit")
    plt.plot(df_logs["t"], df_logs["cum_oracle"], linewidth=2, label="Oracle")
    plt.plot(df_logs["t"], df_logs["cum_single"], linewidth=2, label="Always Single")
    plt.plot(df_logs["t"], df_logs["cum_cut"], linewidth=2, label="Always Cut")
    plt.plot(df_logs["t"], df_logs["cum_teleport"], linewidth=2, label="Always Teleport")
    plt.xlabel("Job index"); plt.ylabel("Cumulative utility"); plt.title("Cumulative utility under drift")
    plt.legend(); plt.tight_layout(); plt.savefig(cumu_png, dpi=160); plt.close()

# ------------------------------
# Optional NVQLink toy
# ------------------------------
def nvqlink_demo(iters=40, q_time=0.02, ms=5.0, us=5.0):
    t=np.arange(iters)
    cum_ms=np.cumsum([q_time+ms/1000.0]*iters)
    cum_us=np.cumsum([q_time+us/1_000_000.0]*iters)
    return t, cum_ms, cum_us

def plot_nvqlink(t, cms, cus, path_png):
    plt.figure(figsize=(7,4))
    plt.plot(t, cms, linewidth=2, label="Feedback 5 ms")
    plt.plot(t, cus, linewidth=2, label="Feedback 5 µs")
    plt.xlabel("VQE iterations"); plt.ylabel("Cumulative wall-clock [s]")
    plt.title("NVQLink-style latency advantage (toy)")
    plt.legend(); plt.tight_layout(); plt.savefig(path_png, dpi=160); plt.close()

# ------------------------------
# Micro-benchmark & smoke grid
# ------------------------------
def micro_benchmark(depth, n_qubits=N_QUBITS, p2q=0.01, L_s=1e-4, reps=1):
    # warm-up (pop caches)
    fidelity_single(p2q, depth); latency_single(depth)
    fidelity_cut(p2q, depth, 1);  latency_cut(depth, 1)
    fidelity_teleport(p2q, depth, L_s); latency_teleport(depth, L_s)

    def timeit(fn, *args):
        t0 = perf_counter()
        for _ in range(reps): fn(*args)
        return (perf_counter() - t0) / reps

    t_fs = timeit(fidelity_single, p2q, depth)
    t_ls = timeit(latency_single, depth)
    t_fc = timeit(fidelity_cut, p2q, depth, 1)
    t_lc = timeit(latency_cut, depth, 1)
    t_ft = timeit(fidelity_teleport, p2q, depth, L_s)
    t_lt = timeit(latency_teleport, depth, L_s)

    per_point = t_fs + t_ls + t_fc + t_lc + t_ft + t_lt
    print("\n=== Micro-benchmark (averaged) ===")
    print(f"fidelity_single  : {t_fs:.3f} s")
    print(f"latency_single   : {t_ls:.3f} s")
    print(f"fidelity_cut     : {t_fc:.3f} s")
    print(f"latency_cut      : {t_lc:.3f} s")
    print(f"fidelity_teleport: {t_ft:.3f} s")
    print(f"latency_teleport : {t_lt:.3f} s")
    print(f"Approx cost per grid-point: {per_point:.3f} s")
    return per_point

# ------------------------------
# Main
# ------------------------------
if __name__ == "__main__":
    print(f"Preset: FAST={FAST}, N_QUBITS={N_QUBITS}, DEPTH={DEPTH}, "
          f"grid={len(P2Q_GRID)}x{len(L_GRID)} ({len(P2Q_GRID)*len(L_GRID)} pts)")

    # 0) Quick smoke test (3x3) to verify pipeline & produce a quick figure
    _sp = np.linspace(P2Q_GRID[0], P2Q_GRID[-1], 3)
    _sl = np.logspace(np.log10(L_GRID[0]), np.log10(L_GRID[-1]), 3)
    t0 = perf_counter()
    df_smoke = decision_boundary_grid(depth=DEPTH, p2q_grid=_sp, L_grid=_sl)
    plot_decision_boundary(df_smoke, f"{OUTDIR}/decision_boundary_smoke.png", smooth_majority=True)
    t1 = perf_counter()
    print(f"Smoke grid 3x3: {t1 - t0:.2f}s -> {OUTDIR}/decision_boundary_smoke.png")

    # 1) Full decision boundary (per preset)
    t0 = perf_counter()
    df_grid = decision_boundary_grid(depth=DEPTH, p2q_grid=P2Q_GRID, L_grid=L_GRID)
    df_grid.to_csv(f"{OUTDIR}/decision_boundary.csv", index=False)
    plot_decision_boundary(df_grid, f"{OUTDIR}/decision_boundary.png", smooth_majority=True)
    t1 = perf_counter()
    print(f"Decision boundary: {t1 - t0:.2f}s -> {OUTDIR}/decision_boundary.png")

    # 2) Adaptation under drift
    t0 = perf_counter()
    df_true = simulate_true_utils(T=40, depth=DEPTH, L_s=1e-3)
    df_logs = run_bandit(df_true, switch_cost=0.001, obs_std=0.01)
    plot_adaptation(df_true, df_logs, f"{OUTDIR}/adaptation_modes.png", f"{OUTDIR}/adaptation_cumulative.png")
    t1 = perf_counter()
    print(f"Adaptation plots: {t1 - t0:.2f}s -> {OUTDIR}/adaptation_*.png")

    # 3) NVQLink toy
    t0 = perf_counter()
    t,cms,cus = nvqlink_demo(iters=40, q_time=0.02, ms=5.0, us=5.0)
    plot_nvqlink(t,cms,cus,f"{OUTDIR}/nvqlink_latency_demo.png")
    t1 = perf_counter()
    print(f"NVQLink plot: {t1 - t0:.2f}s -> {OUTDIR}/nvqlink_latency_demo.png")

    # 4) Optional micro-benchmark printout
    _ = micro_benchmark(depth=DEPTH, n_qubits=N_QUBITS, p2q=0.01, L_s=1e-4, reps=1)

    print("Artifacts written to:", OUTDIR)


Preset: FAST=True, N_QUBITS=3, DEPTH=24, grid=12x14 (168 pts)


/var/folders/bc/nklc6z7d589gm23h8f4952d40000gq/T/ipykernel_39432/2168738483.py:304: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, _ in scheduled.data:


Smoke grid 3x3: 3.11s -> ./artifacts/decision_boundary_smoke.png


/var/folders/bc/nklc6z7d589gm23h8f4952d40000gq/T/ipykernel_39432/2168738483.py:304: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, _ in scheduled.data:


Decision boundary: 41.47s -> ./artifacts/decision_boundary.png


/var/folders/bc/nklc6z7d589gm23h8f4952d40000gq/T/ipykernel_39432/2168738483.py:304: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, _ in scheduled.data:


Adaptation plots: 10.43s -> ./artifacts/adaptation_*.png
NVQLink plot: 0.06s -> ./artifacts/nvqlink_latency_demo.png

=== Micro-benchmark (averaged) ===
fidelity_single  : 0.076 s
latency_single   : 0.000 s
fidelity_cut     : 0.076 s
latency_cut      : 0.000 s
fidelity_teleport: 0.076 s
latency_teleport : 0.000 s
Approx cost per grid-point: 0.229 s
Artifacts written to: ./artifacts
